In [3]:
%reset -f

In [3]:
import pandas as pd
from datetime import datetime
from querys import get_cuenta_npl
from load import export_dataframe_to_sql
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
current_month_year = datetime.now().strftime("%m.%Y")
source = "payments.xlsx"

In [5]:
df = pd.read_excel(source, sheet_name=current_month_year, header=0, usecols="A,D,E,G,H,J")
df = df.dropna(how='all', axis=1).dropna(how='all').reset_index(drop=True)
df.columns = ['FECHA', 'PAGO', 'BANCO', 'CONTRATO', 'DNI', 'CARTERA']
df = df[df['DNI'] != 0]
df['CONTRATO'] = df['CONTRATO'].apply(lambda x: '{:.0f}'.format(x) if pd.notnull(x) else x)
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y', errors='coerce')
df['PAGO'] = df['PAGO'].round(2)

In [6]:
# Limpia espacios y ajusta la longitud de la columna 'CONTRATO'
df['CONTRATO'] = df['CONTRATO'].str.strip()  # Quitar espacios al inicio y al final
df['CONTRATO'] = df['CONTRATO'].apply(lambda x: x.zfill(10))  # Agregar ceros a la izquierda hasta 10 caracteres

In [7]:
df['CUENTA'] = df['CONTRATO'].apply(lambda contrato: get_cuenta_npl(contrato))

In [8]:
export_dataframe_to_sql(df, "precarga", schema='omni')

Datos exportados exitosamente a la tabla 'precarga'.
